In [ ]:
import os
import subprocess

def get_video_ids_with_status(youtube_url):
    """Obtém os IDs de todos os vídeos na playlist, incluindo os indisponíveis."""
    try:
        result = subprocess.run(
            ["yt-dlp", "--flat-playlist", "--print", "%(id)s\t%(title)s", youtube_url],
            capture_output=True,
            encoding="utf-8",
            check=True
        )
        videos = result.stdout.strip().split("\n")
        video_status = {}
        for video in videos:
            parts = video.split("\t")
            if len(parts) == 2:
                video_id, title = parts
                video_status[video_id] = title
            else:
                video_status[video] = "Indisponível ou Deletado"
        return video_status
    except Exception as e:
        print(f"Erro ao obter os IDs e status dos vídeos: {e}")
        return {}

def download_video(video_id, save_path):
    """Faz o download de um vídeo específico."""
    try:
        print(f"Baixando vídeo com ID: {video_id}...")
        command = [
            "yt-dlp",
            "-o", f"{save_path}/%(title)s [%(id)s].%(ext)s",  # Nome com ID para evitar duplicados
            f"https://www.youtube.com/watch?v={video_id}"
        ]
        subprocess.run(command, check=True)
        print(f"Vídeo {video_id} baixado com sucesso!")
    except Exception as e:
        print(f"Erro ao baixar o vídeo {video_id}: {e}")

def find_missing_videos(video_status, save_path):
    """Identifica os vídeos que ainda não foram baixados e estão disponíveis."""
    missing_videos = {}
    for video_id, title in video_status.items():
        video_file = os.path.join(save_path, f"{title} [{video_id}].mp4")
        if not os.path.exists(video_file):
            if "Indisponível" not in title:
                missing_videos[video_id] = title
    return missing_videos

def download_playlist(youtube_url, save_path):
    """Gerencia o download de toda a playlist."""
    try:
        print("Obtendo informações da playlist...")
        video_status = get_video_ids_with_status(youtube_url)
        total_videos = len(video_status)
        print(f"Total de vídeos na playlist: {total_videos}")

        unavailable_videos = {
            video_id: title for video_id, title in video_status.items() if "Indisponível" in title
        }
        print(f"Vídeos indisponíveis: {len(unavailable_videos)}")

        while True:
            missing_videos = find_missing_videos(video_status, save_path)
            if not missing_videos:
                print("Todos os vídeos disponíveis foram baixados com sucesso!")
                break

            print(f"Vídeos restantes para baixar: {len(missing_videos)}")
            for video_id, title in missing_videos.items():
                download_video(video_id, save_path)

        print("\nResumo:")
        print(f"Total de vídeos: {total_videos}")
        print(f"Vídeos indisponíveis ou deletados: {len(unavailable_videos)}")
        print(f"Vídeos baixados com sucesso: {total_videos - len(unavailable_videos)}")

    except Exception as e:
        print(f"Erro ao processar a playlist: {e}")

if __name__ == "__main__":
    # URL da playlist do YouTube
    youtube_url = input("Digite a URL da playlist do YouTube: ")

    # Diretório para salvar os vídeos
    save_path = input("Digite o diretório para salvar os vídeos (exemplo: C:/Videos): ")

    # Garante que o diretório existe
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    download_playlist(youtube_url, save_path)
